# Experiment 007: Asymmetric Placement Exploration

The discussion 'Why the winning solutions will be Asymmetric' (40 votes) suggests asymmetric solutions outperform symmetric ones. Let's analyze the baseline symmetry and try asymmetric perturbations.

In [ ]:
import math
import random
import pandas as pd
import numpy as np
import json
import time
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
from collections import defaultdict

# Tree polygon vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, deg):
    poly = Polygon(zip(TX, TY))
    rotated = affinity.rotate(poly, deg, origin=(0, 0))
    return affinity.translate(rotated, x, y)

def has_any_overlap(trees):
    if len(trees) <= 1:
        return False
    polys = [create_tree_polygon(t[0], t[1], t[2]) for t in trees]
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if polys[i].intersects(polys[j]) and not polys[i].touches(polys[j]):
                if polys[i].intersection(polys[j]).area > 1e-15:
                    return True
    return False

def calculate_score(trees, n):
    if not trees:
        return float('inf')
    polys = [create_tree_polygon(t[0], t[1], t[2]) for t in trees]
    bounds = unary_union(polys).bounds
    side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
    return side ** 2 / n

print("Functions defined")

In [ ]:
# Load baseline configurations
def load_baseline():
    baseline_path = '/home/code/experiments/002_valid_baseline/submission.csv'
    df = pd.read_csv(baseline_path)
    
    configs = defaultdict(list)
    for _, row in df.iterrows():
        n = int(row['id'].split('_')[0])
        x = float(str(row['x']).replace('s', ''))
        y = float(str(row['y']).replace('s', ''))
        deg = float(str(row['deg']).replace('s', ''))
        configs[n].append([x, y, deg])
    
    return dict(configs)

baseline_configs = load_baseline()
print(f"Loaded baseline with {len(baseline_configs)} N values")

# Calculate baseline scores
baseline_scores = {}
for n in range(1, 201):
    if n in baseline_configs:
        baseline_scores[n] = calculate_score(baseline_configs[n], n)

print(f"Baseline total: {sum(baseline_scores.values()):.6f}")

In [ ]:
# Analyze baseline symmetry for selected N values
print("Analyzing baseline symmetry...")
print("="*60)

def analyze_symmetry(config, n):
    """Analyze symmetry properties of a configuration."""
    trees = config
    
    # Calculate centroid
    cx = sum(t[0] for t in trees) / n
    cy = sum(t[1] for t in trees) / n
    
    # Check reflection symmetry (x-axis)
    x_symmetric = True
    for t in trees:
        # Check if there's a tree at reflected position
        reflected_y = 2 * cy - t[1]
        found = False
        for t2 in trees:
            if abs(t2[0] - t[0]) < 0.01 and abs(t2[1] - reflected_y) < 0.01:
                found = True
                break
        if not found:
            x_symmetric = False
            break
    
    # Check rotation angles distribution
    angles = [t[2] % 360 for t in trees]
    unique_angles = set(round(a, 1) for a in angles)
    
    # Check if positions form a regular pattern
    x_coords = sorted([t[0] for t in trees])
    y_coords = sorted([t[1] for t in trees])
    
    return {
        'centroid': (cx, cy),
        'x_symmetric': x_symmetric,
        'unique_angles': len(unique_angles),
        'angle_distribution': unique_angles,
        'x_spread': max(x_coords) - min(x_coords),
        'y_spread': max(y_coords) - min(y_coords)
    }

for n in [5, 10, 20, 50, 100]:
    sym = analyze_symmetry(baseline_configs[n], n)
    print(f"\nN={n}:")
    print(f"  Centroid: ({sym['centroid'][0]:.4f}, {sym['centroid'][1]:.4f})")
    print(f"  X-symmetric: {sym['x_symmetric']}")
    print(f"  Unique angles: {sym['unique_angles']}")
    print(f"  Angle distribution: {sorted(sym['angle_distribution'])}")
    print(f"  Spread: x={sym['x_spread']:.4f}, y={sym['y_spread']:.4f}")

In [ ]:
# Try asymmetric perturbations starting from baseline
print("\nTrying asymmetric perturbations...")
print("="*60)

def asymmetric_perturbation(config, n, seed=None):
    """Apply asymmetric perturbations to break symmetry."""
    if seed is not None:
        random.seed(seed)
    
    trees = [list(t) for t in config]
    
    # Apply directional bias (asymmetric)
    bias_x = random.uniform(-0.05, 0.05)
    bias_y = random.uniform(-0.05, 0.05)
    
    for i in range(n):
        # Position perturbation with directional bias
        trees[i][0] += bias_x + random.gauss(0, 0.01)
        trees[i][1] += bias_y + random.gauss(0, 0.01)
        # Angle perturbation
        trees[i][2] = (trees[i][2] + random.uniform(-3, 3)) % 360
    
    return trees

# Test on small N first
improvements = []
for n in range(2, 21):
    baseline_score = baseline_scores[n]
    best_score = baseline_score
    best_config = None
    
    # Try multiple random seeds
    for seed in range(100):
        perturbed = asymmetric_perturbation(baseline_configs[n], n, seed=seed)
        
        if not has_any_overlap(perturbed):
            score = calculate_score(perturbed, n)
            if score < best_score - 1e-8:
                best_score = score
                best_config = perturbed
    
    if best_config is not None:
        improvement = baseline_score - best_score
        improvements.append((n, improvement))
        print(f"✅ N={n}: {baseline_score:.6f} -> {best_score:.6f} (improved by {improvement:.6f})")
    else:
        print(f"❌ N={n}: {baseline_score:.6f} (no improvement)")

print("="*60)
if improvements:
    print(f"Found {len(improvements)} improvements!")
    print(f"Total improvement: {sum(imp for _, imp in improvements):.6f}")
else:
    print("No improvements found through asymmetric perturbation")

In [ ]:
# Try global rotation optimization
# Rotate ALL trees together to find better bounding box orientation
print("\nTrying global rotation optimization...")
print("="*60)

def rotate_all_trees(config, angle):
    """Rotate all trees around the centroid by given angle."""
    n = len(config)
    trees = [list(t) for t in config]
    
    # Calculate centroid
    cx = sum(t[0] for t in trees) / n
    cy = sum(t[1] for t in trees) / n
    
    # Rotate each tree position around centroid
    rad = math.radians(angle)
    cos_a, sin_a = math.cos(rad), math.sin(rad)
    
    for i in range(n):
        # Translate to origin
        x = trees[i][0] - cx
        y = trees[i][1] - cy
        # Rotate
        new_x = x * cos_a - y * sin_a
        new_y = x * sin_a + y * cos_a
        # Translate back
        trees[i][0] = new_x + cx
        trees[i][1] = new_y + cy
        # Also rotate the tree's own angle
        trees[i][2] = (trees[i][2] + angle) % 360
    
    return trees

# Test global rotation on selected N values
rotation_improvements = []
for n in [10, 20, 30, 50, 75, 100, 150, 200]:
    baseline_score = baseline_scores[n]
    best_score = baseline_score
    best_angle = 0
    
    # Try different rotation angles
    for angle in np.arange(0, 360, 0.5):  # 0.5 degree increments
        rotated = rotate_all_trees(baseline_configs[n], angle)
        
        if not has_any_overlap(rotated):
            score = calculate_score(rotated, n)
            if score < best_score - 1e-8:
                best_score = score
                best_angle = angle
    
    if best_angle != 0:
        improvement = baseline_score - best_score
        rotation_improvements.append((n, improvement, best_angle))
        print(f"✅ N={n}: {baseline_score:.6f} -> {best_score:.6f} (angle={best_angle:.1f}°, improved by {improvement:.6f})")
    else:
        print(f"❌ N={n}: {baseline_score:.6f} (no improvement from rotation)")

print("="*60)
if rotation_improvements:
    print(f"Found {len(rotation_improvements)} improvements from rotation!")
else:
    print("No improvements found through global rotation")

In [ ]:
# Save results
print("\nSaving results...")

import shutil
import os

# Since no improvements were found, use baseline
shutil.copy('/home/code/experiments/002_valid_baseline/submission.csv',
            '/home/code/experiments/007_asymmetric_exploration/submission.csv')
os.makedirs('/home/submission', exist_ok=True)
shutil.copy('/home/code/experiments/002_valid_baseline/submission.csv',
            '/home/submission/submission.csv')

# Save metrics
metrics = {
    'cv_score': sum(baseline_scores.values()),
    'asymmetric_improvements': len(improvements),
    'rotation_improvements': len(rotation_improvements),
    'notes': 'Analyzed baseline symmetry and tried asymmetric perturbations and global rotation optimization. No improvements found - baseline is already well-optimized.'
}

with open('/home/code/experiments/007_asymmetric_exploration/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"CV Score: {metrics['cv_score']:.6f}")
print(f"Asymmetric improvements: {metrics['asymmetric_improvements']}")
print(f"Rotation improvements: {metrics['rotation_improvements']}")
print("Metrics saved")